In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import ftfy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [2]:
def ngrams(string, n=3):
    
    # string = re.sub(r'[,-./]|\sBD',r'', string)
    # ngrams = zip(*[string[i:] for i in range(n)])
    # return [''.join(ngram) for ngram in ngrams]
    
    
    string = ftfy.fix_text(string)
    string = string.encode("ascii", errors="ignore").decode()
    string = string.lower()
    chars_to_remove = [")","(",">","|","{","}","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',',' ')
    string = string.replace('-', ' ')
    string = string.replace("isd", "intermediate school district")
    string = string.replace("usd", "unified school district")
    string = string.replace("sd", "school district")
    string = string.replace("schools", "")
    string = string.replace("school", "")
    string = string.replace("public", "")    
    string = string.replace("districts", "")
    string = string.replace("district", "")
    string = string.replace("independent", "")
    string = string.replace("unified", "")
    string = string.replace("metropolitan", "")
    string = string.replace("academy", "")
    string = string.replace("department", "")
    string = string.replace("board", "")
    string = string.replace("education", "")
    
    

    
    string = string.title()
    string = re.sub(' +',' ',string).strip()
    string = ' '+ string + ' '
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [91]:
# read in the dataset with goal data?? (training data)
clean_org_names = pd.read_csv(r"C:\Users\jacob\Documents\SVS\Contacts to Accounts\Data\Accounts 6.28.22.csv")
clean_org_names = clean_org_names.iloc[:, 0:4]
# insert column name of column with training values
org_name_clean = clean_org_names['Account Name']#.unique()


print('Vecorizing the data - this could take a few minutes for large datasets...')
vectorizer = TfidfVectorizer(min_df=1, lowercase=False, analyzer=ngrams)
tfidf = vectorizer.fit_transform(org_name_clean)
print('Vecorizing completed...')

# read in the data to be acted upon (forward pass)
df_c = pd.read_excel(r"C:\Users\jacob\Documents\SVS\Contacts to Accounts\Data\EduFresh contacts 6.28.22.xlsx")
# enter your column header for the subset
df_c.dropna(subset='School District/Organization',inplace=True)

Vecorizing the data - this could take a few minutes for large datasets...
Vecorizing completed...


In [92]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
org_column = 'School District/Organization' #column to match against in the messy data
unique_org = df_c[org_column].values # set used for increased performance:

### matching query
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

import time
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_org)
t = time.time()-t1
print("COMPLETED IN:", t)
unique_org = list(unique_org) #need to convert back to a list

print('finding matches...')
matches = []
for i,j in enumerate(indices):
    temp = [df_c['Contact ID'].values[i],round(1-distances[i][0],3), clean_org_names.values[j][0][0], clean_org_names.values[j][0][3],unique_org[i]]
    matches.append(temp)
print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Contact ID','accuracy','Record Id','Account Match','Contacts'])
print('Done')

getting nearest n...
COMPLETED IN: 0.12380194664001465
finding matches...
Building data frame...
Done


In [93]:
print("\'wrong\' count: " + str(len(matches.query('accuracy < .18').sort_values("accuracy", ascending=False))))

print("accuracy: "+ str(round((1-len(matches.query('accuracy < .18').sort_values("accuracy", ascending=False))/len(matches))*100, 2))+"%")

'wrong' count: 245
accuracy: 67.03%


In [94]:
a = np.array(matches['Record Id'].values.tolist())
matches['Record Id'] = np.where(matches.accuracy < .3, np.nan, a)
matches['Record Id'] = matches['Record Id'].replace("nan", np.nan)

b = np.array(matches['Account Match'].values.tolist())
matches['Account Match'] = np.where(matches.accuracy < .3, np.nan, b)
matches['Account Match'] = matches['Account Match'].replace("nan", np.nan)

In [95]:
matches.sort_values('accuracy', ascending = True).head(10)

,Contact ID,accuracy,Record Id,Account Match,Contacts
725,zcrm_5090086000002614561,-0.277,NaN,NaN,JoyceVentures LLC Business Strategy for Social Good
655,zcrm_5090086000002614285,-0.270,NaN,NaN,Ashbeck Behavior Consulting LLC
483,zcrm_5090086000002614113,-0.266,NaN,NaN,Catholic Education Melbourne
681,zcrm_5090086000002614517,-0.260,NaN,NaN,Renaissance Academy of Multimedia Arts
353,zcrm_5090086000001583594,-0.258,NaN,NaN,Creatively Focused
110,zcrm_5090086000001290007,-0.247,NaN,NaN,Let Them Be Kids!
401,zcrm_5090086000001675074,-0.244,NaN,NaN,Asotin-Anatone School District
96,zcrm_5090086000001281001,-0.243,NaN,NaN,Linaschildcare
189,zcrm_5090086000001583224,-0.234,NaN,NaN,Aliquippa School District
87,zcrm_5090086000001273014,-0.233,NaN,NaN,Itâ€™s a good day preschool


### re-merging

In [96]:
outMatch = matches.drop(['Contacts'], axis = 1)

In [97]:
cols = outMatch.columns.tolist()

In [98]:
cols = ['Contact ID', 'Record Id', 'accuracy', 'Account Match']

In [99]:
outMatch = outMatch[cols]

In [100]:
df_c = pd.read_excel(r"C:\Users\jacob\Documents\SVS\Contacts to Accounts\Data\EduFresh contacts 6.28.22.xlsx")

In [101]:
df = pd.merge(left=df_c, right=outMatch, on='Contact ID', how='outer',suffixes=["", "_Accounts"])

In [102]:
df.to_csv("Final Contact - Account Integration.csv")